Generates the stock news graph.

In [5]:
import pandas as pd
from config import * 
import yfinance as yf 
import plotly as py
import plotly.graph_objects as go
from datetime import datetime

stock = 'GOOGL'
df = pd.read_csv(f'{OUTPUT_STOCK_PATH}/{stock}.csv', parse_dates=['Date'])

def n_largest(col, n):
    return col.dropna().sort_values(key=abs, ascending=False).unique()[:n]

largest_price_changes = n_largest(df['% Change'], n = 2)
print(largest_price_changes)

n=2
catalysts = df.loc[df.groupby('% Change')['hits'].idxmax().dropna()]
catalysts['Date'] = catalysts['Date'].dt.normalize()        # strip out the time part 

catalysts = catalysts.head(n)


[5.59 5.52]


In [6]:
# get stock price action
prices = yf.download(stock, period="1y", interval="1d")
prices.columns = prices.columns.droplevel(1) 
prices.reset_index(inplace = True)
prices.name = stock



[*********************100%***********************]  1 of 1 completed


In [38]:
fig = go.Figure(data=[go.Candlestick(x=prices['Date'],
                open=prices['Open'],
                high=prices['High'],
                low=prices['Low'],
                close=prices['Close'])]
            )

import random 

# Add title and axis labels
fig.update_layout(
    title= "Google",
    xaxis_title="Date",
    yaxis_title="Price",
    yaxis_range=[160,200]
)

# add annotations 
for index, row in catalysts.iterrows():
    print(prices[prices['Date'] == row['Date']]['Open'])
    fig.add_annotation(
        x=row["Date"],  # Date for the note
        y= prices[prices['Date'] == row['Date']]['Open'].iloc[0],
        text=f"{row['Title']}",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=100,          # Offset for the note
        bgcolor="yellow", # Background color for note
        bordercolor="black"
    )




fig.show()
a = 5


216    173.700922
Name: Open, dtype: float64
229    185.309998
Name: Open, dtype: float64


In [37]:

fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()
